In [1]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure 
from bokeh.io import output_notebook, show
from bokeh.transform import factor_cmap
from bokeh.palettes import Category20
from os.path import dirname, join

import numpy as np
import pandas.io.sql as psql
import sqlite3 as sql

from bokeh.plotting import figure
from bokeh.layouts import layout, widgetbox
from bokeh.models import ColumnDataSource, Div
from bokeh.models.widgets import Slider, Select, TextInput, DateRangeSlider, DateSlider, MultiSelect, Button, Toggle, RangeSlider
from bokeh.io import curdoc
from datetime import date, timedelta
from bokeh.models import HoverTool

from time import sleep

output_notebook()

Loading BokehJS ...

# Preprocessing

In [2]:
df = pd.read_csv("AllBirdsv4.csv")
df_test = pd.read_csv("Test Birds Location.csv")

# replace "?" characters
#df = df.replace('?', None)
df["Y"] = df["Y"].map(lambda y: y.replace("?",""))
df["Y"] = df["Y"].astype('int64')
        
# Add a specific color for each specie according to Category20
birds_types = df['English_name'].sort_values().unique()
n_birds_types = len(birds_types)
birds_types_colors = pd.Series(Category20[n_birds_types], index = birds_types).rename('color')
df = df.join(birds_types_colors, on = "English_name")        
df['alpha'] = 1

# Convert date string to datetime
a = pd.to_datetime(df['Date'], errors = 'coerce', format = "%m/%d/%Y").dropna()
b = pd.to_datetime(df['Date'], errors = 'coerce', format = "%Y-%m-%d").dropna()
c = pd.to_datetime(df['Date'], errors = 'coerce', format = "%Y-%m-00").dropna()
df['T'] = pd.concat([a,b,c], axis = 0)

# Only keep valid date
df = df.loc[~df['T'].isna()].copy()

# Birdmap

In [3]:
def bird_map(doc): 

    # Create Input controls
    date_input = DateSlider(title="Date d'observation", start=df['T'].min(), end=df['T'].max(),
                            value=df['T'].min(), step=1, format = "%b %Y")
    # Create Input controls
    speed_input = Slider(title="Vitesse (jours)", start=10, end=100,
                            value=30, step=1)
    T_input = Slider(title="Fenêtre d'observation (jours)", start=30, end=360,
                            value=180, step=1)


    bird_types_input = MultiSelect(title="Espèces", value=["All"],
                   options= ['All'] + list(birds_types) , size = n_birds_types + 1)
    
    button = Toggle(label="Play", button_type="success")
    
    
    
    # Create Column Data Source that will be used by the plot
    source = ColumnDataSource(data=dict(x=[], y=[], color=[], alpha=[], name = []))

    TOOLS="crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

    p = figure(title="Map with birds through time", tools = TOOLS, x_range=(0, 200), y_range=(0, 200))
    # plot map
    p.image_url( url=["map.jpg"],
             x=0, y=0, w=200, h=200, anchor="bottom_left")
    p.circle(x="x", y="y", source=source, size=12, color="color", line_color=None, fill_alpha="alpha")
    
    
    hover = HoverTool()
    hover.tooltips = [
        ("Bird type", "@name"),
        ("Position", "(@x, @y)"),
    ]
    p.tools.append(hover)



    def select_obs(T):
        bird_types = bird_types_input.value
        if "All" not in bird_types:
            selector_types = df['English_name'].isin(bird_types)
        else:
            selector_types = [True] * df.shape[0]

        a = pd.Timestamp(date_input.value - timedelta(days=T))
        b = pd.Timestamp(date_input.value + timedelta(days=T))
        selector_date = ((a < df['T']) & (df['T'] < b))

        return df.loc[selector_date & selector_types] # Selection

    def update():
        T = T_input.value
        
        df_selected = select_obs(T)   
        
        # Alpha dynamic
        t = pd.to_datetime(date_input.value)
        a = (df_selected['T'] - t).dt.days.abs()
        a = 1 - 2*a/T
        a.loc[a < 0] = 0

        source.data = dict(
            x=df_selected['X'],
            y=df_selected['Y'],
            color=df_selected["color"],
            alpha=a,
            name=df_selected["English_name"],
        )
    
    def run_time():
        run_time_on = button.active
        button.button_type = "danger"  if (run_time_on) else "success" # Update color button
        button.label = "Stop"  if (run_time_on) else "Play" # Update color button
        speed = speed_input.value
            
        if(run_time_on):
            a = date_input.value + timedelta(days=speed)
            if pd.Timestamp(a) > pd.Timestamp(date_input.end):
                button.active = False
                date_input.value = date_input.start
            else:
                date_input.value = a
                doc.add_timeout_callback(run_time, 50)
            
            
        

    controls = [bird_types_input, date_input, T_input]
    for control in controls:
        control.on_change('value', lambda attr, old, new: update())
    controls += [button, speed_input]
    button.on_change('active',lambda attr, old, new : run_time())

    sizing_mode = 'fixed'  # 'scale_width' also looks nice with this example

    inputs = widgetbox(*controls, sizing_mode=sizing_mode)
    l = layout([
        [inputs, p],
    ], sizing_mode=sizing_mode)

    update()  # initial load of the data

    doc.add_root(l)

    
show(bird_map)